In [ ]:
# default_exp core

# Powerline Detection

> Use Hough Transform to extract powerlines from LiDAR data.

In [ ]:
#hide
from nbdev.showdoc import *
import pathlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import laspy as lp

In [ ]:
#export
import numpy as np
from tqdm import tqdm
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#export
def grid_subsampling(points, voxel_size):
    "Define a function that takes as input an array of points, and a voxel size expressed in meters"
    "points: it is from a LiDAR data; voxel_size: the length of edge"
    nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)
    print(nb_vox.shape)
    a = ((points - np.min(points, axis=0)) // voxel_size).astype(int)
    non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(a, axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted=np.argsort(inverse)
    voxel_grid={}
    grid_barycenter,grid_candidate_center=[],[]
    last_seen=0
    for idx, vox in enumerate(tqdm(non_empty_voxel_keys)):
        per_voxel = nb_pts_per_voxel[idx]
        num_pvs   = last_seen + per_voxel
        idxs      = idx_pts_vox_sorted[last_seen: num_pvs]
        voxel_grid[tuple(vox)] = points[idxs]
        gbc = np.mean(points[idxs], axis=0)
        grid_barycenter.append(gbc)
        #
        temp = voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0)
        temp = np.linalg.norm(temp, axis=1).argmin()                                             
        grid_candidate_center.append(voxel_grid[tuple(vox)][temp])
        last_seen += per_voxel
    #
    grid_barycenter       = np.array(grid_barycenter)
    grid_candidate_center = np.array(grid_candidate_center)
    return {'barycenter': grid_barycenter, 'candidate': grid_candidate_center}

In [ ]:
#export
class Arrow3D(FancyArrowPatch):
    "it is a FancyArrowPatch Class, which can be used to draw 3D arrow in PCA"
    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0,0),(0,0), *args, **kwargs)
        self._xyz = (x,y,z)
        self._dxdydz = (dx,dy,dz)
    def draw(self, renderer):
        x1,y1,z1 = self._xyz
        dx,dy,dz = self._dxdydz
        x2,y2,z2 = (x1+dx, y1+dy, z1+dz)
                
        xs, ys, zs = proj_transform((x1,x2),(y1,y2),(z1,z2), renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        super().draw(renderer)
#export
def arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

In [ ]:
#export 
def getDirctionVectorsByPCA(pca):
    "calculate the direction vectors of PCA in a point cloud"
    abc = []
    for length, vector in zip(pca.explained_variance_, pca.components_):
        v = vector *  np.sqrt(length)
        abc.append(v.tolist())
    return abc
#export 
def pointsProjectAxis(abc,startPoint, pts):
    "calculate the points that are mapped onto the pca lines from a point cloud"
    '''
    abc: slope vector
    startPoint: starting point of a line
    pts: coordinates of all points
    '''
    m,n,p = abc
    xo,yo,zo = startPoint
    pts_ty=[]
    for i in pts:
        t = (m*(i[0] - xo) + n*(i[1] - yo) + p*(i[2] - zo))/(m**2 + n**2 + p**2)
        xc = m*t + xo
        yc = n*t + yo
        zc = p*t + zo
        pts_ty.append([xc,yc,zc])
    return pts_ty
#export 
def radiusOfCylinderByLeastSq(**kwargs):
    "use least square algorithm to calculate the radius of a cylinder"
    pts = kwargs['points'] # the coordinates of points
    projs = kwargs['projections'] # the points that are mapped onto the pca lines from a point cloud
    a = pts - projs
    dist=np.array([norm(i) for i in a])
    errfunc = lambda cr:np.sum((dist - cr)**2)
    est_p, success = leastsq(errfunc,0)
    return np.ceil(est_p)
#export 
def get_start_end_line(pts):
    "get the starting point and end point of a line"
    px,py = pts[:,0], pts[:,1]
    idx = np.lexsort((py,px))
    start = pts[idx[0]]
    end = pts[idx[-1]]
    return np.array([start,end])
#export 
def cylinderSurface(**kwargs):
    "generate the surface of a cylinder"
    pts = kwargs['points'] # points in the axis of a cylinder
    R = kwargs['radius']
    pca = kwargs['pca']
    p0, p1 = get_start_end_line(pts)
    mag = norm(p0 - p1)
    #vector in direction of axis
    v,n1,n2 = pca.components_
    # judge whether the direction of u and v is same
    u = p1 - p0
    j = np.dot(v,u)/(norm(u)*norm(v))
    if j < 0:
        p0, p1 = (p1,p0)
    #surface ranges over t from 0 to length of axis and 0 to 2*pi
    t = np.linspace(0, mag, 100)
    theta = np.linspace(0, 2 * np.pi, 100)
    #use meshgrid to make 2d arrays
    t, theta = np.meshgrid(t, theta)
    #generate coordinates for surface
    cX, cY, cZ = [p0[i] + v[i] * t + R * np.sin(theta) * n1[i] + R * np.cos(theta) * n2[i] for i in [0, 1, 2]]
    return (cX, cY, cZ)
#export   
def extractFeathersByPointCloud(**kwargs):
    "calculate the axis length sphericity, linarity and planarity of a point cloud"
    pca = kwargs['pca']
    pts = kwargs['points']
    abc = getDirctionVectorsByPCA(pca)
    pts_lamda_0 = pointsProjectAxis(abc[0],pca.mean_,pts)
    a = {'points': pts, 'projections': pts_lamda_0}
    r = radiusOfCylinderByLeastSq(**a)
    pts_lamda_0 = np.array(pts_lamda_0)
    p0, p1 = get_start_end_line(pts_lamda_0)
    lamda1, lamda2, lamda3 = np.sqrt(pca.explained_variance_)
    #
    axis_length = norm(p0 - p1)
    sp = lamda3/lamda1
    ln = (lamda1 - lamda2)/lamda1
    pl = (lamda2 - lamda3)/lamda1
    rtn = {'radius': r[0], 'axis': axis_length, 'sphericity': sp, 'linearity': ln, 'planarity': pl}
    return rtn
 

# Hough Transform

In [ ]:
#export
def pointsToRaster(**kwargs):
    "Transform a point cloud to a raster image"
    xyz_min = kwargs['min_xyz']
    xyz_max = kwargs['max_xyz']
    pts = kwargs['pts']
    cellsize = kwargs['cellsize']
    #
    xyz_min = np.floor(xyz_min)
    xyz_max = np.floor(xyz_max)
    cell_ids = (pts - xyz_min)/cellsize
    cell_ids = np.floor(cell_ids)
    cell_ids = np.unique(cell_ids,axis=0) # eliminate the replicated values
    cell_ids = cell_ids.astype(int)
    #
    a = np.floor((xyz_max - xyz_min)/cellsize)
    a = a.astype(int)
    xnum = a[0] + 1
    ynum = a[1] + 1
    img_xy = np.zeros((ynum, xnum), dtype='uint8')
    for m in cell_ids:
        i,j,k = m
        img_xy[j,i] = 1
    return img_xy
#export
def getCellIDByPolarCoordinates(**kwargs):
    "Get the indices of cells according to polar coordinates"
    '''
    Modifyed by Haitao Lyu on 08/08/2020
    Input Argument:
        houghPic: a 2D matrix that represents a raster picture, which generated by the function 'pointsToRaster'.
        pts: a polar coordinates list that comes from the return value of the function'houghToRasterByNumOfCrossLines'
        threshold: the maximal distance between a cell and a line
    Return Value:
        rtn: a list that includes the index of row and num and the corresponding slope angle
    '''
    houghPic = kwargs['img']
    peaks = kwargs['peaks']
    threshold = kwargs['maxdist']
    #
    rtn=[]
    oy,ox = np.nonzero(houghPic)
    for nc, angle, dist in peaks:
        a = np.math.cos(angle)
        b = np.math.sin(angle)
        c = -dist
        d_line = np.abs(a*ox + b*oy + c)/np.sqrt(a*a + b*b)
        #
        d_l = d_line[d_line<threshold]
        if len(d_l) < 1:
            
            print(f'this line length:{len(d_l)}, neglect it')
            continue
        #
        nc = np.int(nc)
        #
        if nc >= len(d_l):
            nc = len(d_l) -1
        #
        e = np.partition(d_l, nc)
        e = e[:nc]
        line = []
        for i in e:
            idx = np.argwhere(d_line == i)
            idx = idx.ravel()
            row = oy[idx]
            col = ox[idx]
            rc = np.vstack((row,col)).transpose()
            t = rc.tolist()
            #t = [row, col, angle, dist]
            line = line + t
        #
        line = np.unique(line, axis=0)
        line = line.tolist()
        rtn.append(line)
    return rtn
    pass

#export
def houghToRasterByCellID(**kwargs):
    "Transform the result of Hough transform to a raster image"
    rc = getCellIDByPolarCoordinates(**kwargs)
    houghPic = kwargs['img']
    img_temp = np.zeros_like(houghPic)
    for ij in rc:
        ij = np.transpose(ij)
        i = ij[0,:]
        j = ij[1,:]
        img_temp[i,j] = 1
    return img_temp
    pass
#export
def recordHoughLines(**kwargs):
    "encapsulate each hough line into a class HLine and save all hough lines in an array"
    '''
    Modifyed by Haitao Lyu on 08/08/2020
    Description: encapsulate each hough line into a class HLine and save all hough lines in an array
    Parameters:
        houghpic: a 2D matrix that represents a raster picture
        houghpeaks: generated by hough_line_peaks, which is a function of library skimage
        threshold: the maximal distance between a cell and a line
    Return:
        an array including hough line encapsulated by class HLine
    '''
    houghPic = kwargs['img']
    peaks = kwargs['peaks']
    threshold = kwargs['maxdist']
    rtn = []
    oy,ox = np.nonzero(houghPic)
    for nc, angle, dist in peaks:
        a = np.math.cos(angle)
        b = np.math.sin(angle)
        c = -dist
        d_line = np.abs(a*ox + b*oy + c)/np.sqrt(a*a + b*b)
        #
        d_l = d_line[d_line<threshold]
        if len(d_l) < 1: 
            continue
        #
        nc = len(d_l) -1
        e = np.partition(d_l, nc)
        e = e[:nc]
        line =[]
        coord = [nc, angle, dist]
        for i in e:
            idx = np.argwhere(d_line == i)
            idx = idx.ravel()
            row = oy[idx]
            col = ox[idx]
            rc = np.vstack((row,col)).transpose()
            t = rc.tolist()
            line = line + t
        #
        line = np.unique(line, axis=0)
        HL = Hline(coord, line, angle,dist)
        rtn.append(HL)
    return rtn
#
#export 
def countNumOfConsecutiveObj(*args):
    "count the number of consecutive 0"
    ary = args[0]
    obj = args[1]
    if len(ary) < 0:
        return -1
    a = 0
    cun = []
    for i in ary:
        if i == obj:
            a = a + 1
        else:
            cun.append(a)
            a = 0
    return cun
#export
def calRowByColViaLineEquation(**kwargs):
    "compute row:y according to col:x based on a line equation"
    '''
    Parameters:
        hl: it is self-defined class HLine, which saves the details of a line
    '''
    hl = kwargs['HLine']
    threshold = kwargs['maxdist']
    angle = hl.angle
    dist = hl.dist
    row_col_min = np.min(hl.line, axis=0)
    row_col_max = np.max(hl.line, axis=0)
    r = np.arange(row_col_min[0], row_col_max[0] + 1)
    c = np.arange(row_col_min[1], row_col_max[1] + 1)
    x,y = np.meshgrid(c,r)
    ox = x.ravel()
    oy = y.ravel()
    a = np.math.cos(angle)
    b = np.math.sin(angle)
    c = -dist
    d_line = np.abs(a*ox + b*oy + c)/np.sqrt(a*a + b*b)
    idx = d_line < threshold
    row = oy[idx]
    col = ox[idx]
    rc = np.vstack((row,col)).transpose()
    rc = np.unique(rc, axis=0)
    #t = rc.tolist()
    return rc
#
#export 
def generateCorridorByLine(**kwargs):
    "generate a powerline corridor in a raster image "
    houghPic = kwargs['img']
    hl = kwargs['HLine']
    img_temp = np.zeros_like(houghPic)
    if len(hl) < 0:
        return -1
    elif len(hl) == 1:
        hl = hl[0]
        for rc in hl.line:
            i = rc[0]
            j = rc[1]
            img_temp[i,j] = 1
    else:
        for h in hl:
            for rc in h.line:
                i = rc[0]
                j = rc[1]
                img_temp[i,j] = 1
    return img_temp
#export
def generateBuffer(**kwargs):
    "generate buffer zone based on the hough transform"
    '''
    Description: generate buffer zone based on the hough transform
    '''
    houghPic = kwargs['img']
    peaks = kwargs['peaks']
    threshold = kwargs['maxdist']
    #
    img = np.ones_like(houghPic)
    oy,ox = np.nonzero(img)
    img_temp = np.zeros_like(houghPic)
    for nc, angle, dist in peaks:
        a = np.math.cos(angle)
        b = np.math.sin(angle)
        c = -dist
        d_line = np.abs(a*ox + b*oy + c)/np.sqrt(a*a + b*b)
        #
        d_l = d_line[d_line<threshold]
        if len(d_l) < 1: 
            continue
        #
        nc = len(d_l) -1
        e = np.partition(d_l, nc)
        e = e[:nc]
        for i in e:
            idx = np.argwhere(d_line == i)
            row = oy[idx].flatten()
            col = ox[idx].flatten()
            img_temp[row, col] = 1
    return img_temp
#export
def locatePointsFromBuffer(**kwargs):
    "obtain points from original point cloud base on the image buffer"
    '''
    Description: obtain points from original point cloud base on the image buffer 
    '''
    img_buffer = kwargs['img_buffer']
    min_xyz = kwargs['min_xyz']
    pts = kwargs['pts']
    cellsize = kwargs['cellsize']
    #
    ly, lx = np.nonzero(img_buffer)
    x_min, y_min = min_xyz[0:2]
    px = cellsize * lx + x_min
    py = cellsize * ly + y_min
    nid_line=[]
    zipped = zip(px,py)
    for i,j in zipped:
        fcx = np.logical_and(pts[:,0] >= i, pts[:,0] < (i + cellsize))
        fcy = np.logical_and(pts[:,1] >= j, pts[:,1] < (j + cellsize))
        idx = np.logical_and(fcx, fcy)
        a = np.argwhere(idx==True)
        a = a.ravel()
        nid_line = nid_line + a.tolist()
    return nid_line
#export
def locatePointsFromBufferSlideWindow(**kwargs):
    "obtain points from original point cloud base on the image buffer"
    '''
    Description: obtain points from original point cloud base on the image buffer 
    '''
    img_buffer = kwargs['img_buffer']
    min_xyz = kwargs['min_xyz']
    pts = kwargs['pts']
    cellsize = kwargs['cellsize']
    zl,zh = kwargs['z_range']
    #
    ly, lx = np.nonzero(img_buffer)
    x_min, y_min = min_xyz[0:2]
    px = cellsize * lx + x_min
    py = cellsize * ly + y_min
    nid_line=[]
    zipped = zip(px,py)
    for i,j in zipped:
        fcx = np.logical_and(pts[:,0] >= i, pts[:,0] < (i + cellsize))
        fcy = np.logical_and(pts[:,1] >= j, pts[:,1] < (j + cellsize))
        fcz = np.logical_and(pts[:,2] >= zl, pts[:,2] < zh)
        idx = np.logical_and(fcx, fcy)
        idx = np.logical_and(fcz,idx)
        a = np.argwhere(idx==True)
        a = a.ravel()
        nid_line = nid_line + a.tolist()
    return nid_line
#
#export 
def getFeaturesFromPointCloud(**kwargs):
    "obtain the features of each point"
    pca = kwargs['pca']
    n = kwargs['n'] # the number of nearest points
    xyz = kwargs['points']
    #
    tree = KDTree(xyz)
    features = []
    with progressbar.ProgressBar(max_value = xyz.shape[0]) as bar:
        for i in range(xyz.shape[0]):
            di = tree.query(xyz[i], k=n)
            pts = xyz[di[1]]
            pca.fit(pts)
            a = {'points': pts, 'pca': pca}
            fts = extractFeathersByPointCloud(**a)
            r = fts['radius']
            al = fts['axis']
            sp = fts['sphericity']
            ln = fts['linearity']
            pl = fts['planarity']
            features.append([r, al, sp, ln, pl])
            bar.update(i)
    return features

# Self-defined Data Structure

In [ ]:
#export
class LPoints:
    "the class is used to record the points' indices in the original dataset."
    def __init__(self, args):
        '''
        args: a List array including the ID in original data
        '''
        k = [i for i in range(len(args))]
        dic = dict.fromkeys(k)
        for k,v in enumerate(args):
            dic[k] = v
        self.dic = dic
#export
def generateLPByIDS(idx):
    "generate a Lpoints class based on a point list"
    a = np.argwhere(idx == True)
    a = a.T
    nid = a.ravel()
    return LPoints(nid)
#export
class Hline(object):
    "a class is used to record the details of lines extracted by Hough Transform"
    '''
    coord: parameters of a line
    line: detailed coordinates
    '''
    def __init__(self, *args):
        self.coord = args[0]
        self.line = args[1]
        self.angle = args[2]
        self.dist = args[3]
    def getLengthOfLine(self):
        line = self.line
        return len(line)
#export
class ImageBuffer():
    "it is use to record the buffer image details of powerline corridors"
    def __init__(self, **kwargs):
        self.img = kwargs['img']
        self.coord = kwargs['coord']

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
